# Comparison of DFT solvers

We compare four different approaches for solving the DFT minimisation problem,
namely a density-based SCF, a potential-based SCF, direct minimisation and Newton.

First we setup our problem

In [1]:
using DFTK
using LinearAlgebra

a = 10.26  # Silicon lattice constant in Bohr
lattice = a / 2 * [[0 1 1.];
                   [1 0 1.];
                   [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms     = [Si, Si]
positions = [ones(3)/8, -ones(3)/8]

model = model_LDA(lattice, atoms, positions)
basis = PlaneWaveBasis(model; Ecut=5, kgrid=[3, 3, 3])

# Convergence we desire
tol = 1e-12
is_converged = DFTK.ScfConvergenceDensity(tol);

## Density-based self-consistent field

In [2]:
scfres_scf = self_consistent_field(basis; is_converged);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -7.846860416912                   -0.70    4.8
  2   -7.852320327895       -2.26       -1.53    1.0
  3   -7.852646207987       -3.49       -2.52    3.2
  4   -7.852646677558       -6.33       -3.37    2.2
  5   -7.852646686077       -8.07       -4.68    1.5
  6   -7.852646686727       -9.19       -5.37    3.2
  7   -7.852646686730      -11.56       -6.15    1.2
  8   -7.852646686730      -12.81       -7.86    2.2
  9   -7.852646686730      -14.27       -7.73    2.8
 10   -7.852646686730   +  -14.45       -8.48    2.0
 11   -7.852646686730   +    -Inf       -9.09    2.2
 12   -7.852646686730   +  -14.75       -9.98    2.0
 13   -7.852646686730      -14.75      -11.02    2.2
 14   -7.852646686730      -15.05      -11.19    1.5
 15   -7.852646686730      -15.05      -12.06    2.8


## Potential-based SCF

In [3]:
scfres_scfv = DFTK.scf_potential_mixing(basis; is_converged);

n     Energy            log10(ΔE)   log10(Δρ)   α      Diag
---   ---------------   ---------   ---------   ----   ----
  1   -7.846819075201                   -0.70           4.8
  2   -7.852553213346       -2.24       -1.63   0.80    2.0
  3   -7.852639161098       -4.07       -2.72   0.80    1.0
  4   -7.852646512585       -5.13       -3.38   0.80    2.0
  5   -7.852646683028       -6.77       -4.29   0.80    1.5
  6   -7.852646686612       -8.45       -4.89   0.80    1.8
  7   -7.852646686719       -9.97       -5.72   0.80    1.2
  8   -7.852646686730      -10.96       -6.36   0.80    2.0
  9   -7.852646686730      -12.91       -7.31   0.80    1.5
 10   -7.852646686730      -14.05       -7.94   0.80    2.0
 11   -7.852646686730   +    -Inf       -8.97   0.80    1.2
 12   -7.852646686730   +    -Inf      -10.13   0.80    2.8
 13   -7.852646686730      -14.75      -10.38   0.80    2.8
 14   -7.852646686730   +  -14.45      -11.34   0.80    1.0
 15   -7.852646686730      -14.45      -

## Direct minimization

In [4]:
scfres_dm = direct_minimization(basis; tol);

Iter     Function value   Gradient norm 
     0     1.410897e+01     3.877618e+00
 * time: 0.3966841697692871
     1     1.134426e+00     1.958723e+00
 * time: 0.6038470268249512
     2    -1.615841e+00     2.272934e+00
 * time: 0.6271970272064209
     3    -3.581695e+00     2.064738e+00
 * time: 0.6606171131134033
     4    -4.981044e+00     2.038922e+00
 * time: 0.6940851211547852
     5    -6.536462e+00     1.404998e+00
 * time: 0.7273671627044678
     6    -7.205873e+00     7.784236e-01
 * time: 0.7607560157775879
     7    -7.496883e+00     5.826828e-01
 * time: 0.7838292121887207
     8    -7.654754e+00     1.204961e-01
 * time: 0.8068861961364746
     9    -7.726058e+00     1.918430e-01
 * time: 0.8304250240325928
    10    -7.777650e+00     8.176157e-02
 * time: 0.8541011810302734
    11    -7.809755e+00     7.683105e-02
 * time: 0.8777501583099365
    12    -7.837414e+00     6.116649e-02
 * time: 0.900719165802002
    13    -7.846180e+00     3.892607e-02
 * time: 0.92368221282

## Newton algorithm

Start not too far from the solution to ensure convergence:
We run first a very crude SCF to get close and then switch to Newton.

In [5]:
scfres_start = self_consistent_field(basis; tol=1e-1);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -7.846717120236                   -0.70    4.8
  2   -7.852312467576       -2.25       -1.53    1.0


Remove the virtual orbitals (which Newton cannot treat yet)

In [6]:
ψ, _ = DFTK.select_occupied_orbitals(basis, scfres_start.ψ, scfres_start.occupation)
scfres_newton = newton(basis, ψ; tol);

n     Energy            log10(ΔE)   log10(Δρ)
---   ---------------   ---------   ---------
  1   -7.852646686704                   -2.54
  2   -7.852646686730      -10.60       -5.92
  3   -7.852646686730   +    -Inf      -12.58


## Comparison of results

In [7]:
println("|ρ_newton - ρ_scf|  = ", norm(scfres_newton.ρ - scfres_scf.ρ))
println("|ρ_newton - ρ_scfv| = ", norm(scfres_newton.ρ - scfres_scfv.ρ))
println("|ρ_newton - ρ_dm|   = ", norm(scfres_newton.ρ - scfres_dm.ρ))

|ρ_newton - ρ_scf|  = 6.449824040930948e-13
|ρ_newton - ρ_scfv| = 2.416968114532289e-13
|ρ_newton - ρ_dm|   = 9.57069652494818e-10
